# England residential UPRNs : changes from 2020-2024
### Please run the cells in sequence!

Author - Samantha Iacob

Last updated - 15/4/2024

To-do: add map and dashboard controls 


In [ ]:
import panel as pn
pn.extension()
import pandas as pd
pd.set_option('plotting.backend', 'pandas_bokeh')

from bokeh.plotting import figure, output_notebook, show, save
from bokeh.models import ColumnDataSource, HoverTool

output_notebook() #specify the Bokeh plots should be embedded within the Jupyter notebook

In [ ]:
regions = pd.read_csv('e_regions.csv') #importing regional figures




In [ ]:
regions_change = regions.copy() # creating a copy to work on

#adding  averages entries
row_list = []
for year in range (2020,2025): 
    row_list.append({'RGN': 'E92000001', #England's country code
                     'UPRN_count': regions_change[(regions_change.Year == year)].UPRN_count.sum(),
                     'Year': year,
                     'name': 'England'
                    })
#tacking this list onto the end of the dataframe    
for row in row_list:
    regions_change.loc[len(regions_change)] = row
    
regions_change.set_index(regions_change.RGN, inplace = True) # setting index to region code (needed in the line charts)
#adding a change percent from 2020
regions_change['change_pct'] = ((regions_change.UPRN_count / (regions_change[(regions_change.Year == 2020)].loc[regions_change.RGN].UPRN_count) - 1) * 100)

In [ ]:
#setting up some stuff for the lines on the table
region_dict = {'E12000001' : 'North East',
               'E12000002' : 'North West', 
               'E12000003' :'Yorkshire and the Humber', 
               'E12000004' :'East Midlands', 
               'E12000005' :'West Midlands',
               'E12000006' : 'East',
               'E12000007' : 'London', 
               'E12000008' : 'South East', 
               'E12000009' :'South West'}
from files.colors import pault_vibrant as colors # a short list of accessible colors for this chart

In [ ]:
# creating a new plot with a title and axis labels
p = figure(title="% change in residential property counts, 2020-2024, baseline year 2020", 
           x_axis_label='year', 
           y_axis_label='% change in UPRNS (baseline 2020)',
           width=800, 
           height=600,
          )

In [ ]:
#creatings the line renderers because it'll be easier to adapt for future things maybe
counter = 0
for key,value in region_dict.items():
    p.line(regions_change[regions_change.RGN == key].Year, 
           regions_change[regions_change.RGN == key].change_pct,
           name = value,
           legend_label = value, 
           color = colors[counter], 
           line_width = 2
          )
    counter += 1

In [ ]:
#creating a separate average line
p.line(regions_change[regions_change.RGN == 'E92000001'].Year, 
           regions_change[regions_change.RGN == 'E92000001'].change_pct,
           name = 'England',
           legend_label = 'England', 
           color = 'Black', 
           line_width = 2,
           line_dash = 'dashed'
          )

In [ ]:
p.legend.location = "top_left"
# configure the tooltip 
p.add_tools(HoverTool(tooltips=[("Region", "$name"),
                               ("Year", "@x"),
                               ("UPRNS", ) 
                               ("% Change", "@y")
                               ]) ) 

In [ ]:
show(p)